# Lab12-2

## Cost: Sequence_loss

In [9]:
import tensorflow as tf

y_data = tf.constant([[1, 1, 1]])

prediction = tf.constant([[[0.2, 0.7]], [[0.6, 0.2]], [[0.2, 0.9]]], dtype=tf.float32)

weights = tf.constant([[1, 1, 1]], dtype=tf.float32)

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=prediction, targets=y_data, weights=weights)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

print("Loss: ", sequence_loss.eval(session=sess))

Loss:  0.596759


## Cost: sequence_loss2

In [12]:
import tensorflow as tf

# define compare data
y_data = tf.constant([[1, 1, 1]])

# virtual rnn result : prediction data
prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]], dtype=tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]], dtype=tf.float32)

# define weights - 각 자리의 중요도. 거의 1로 셋팅한다.
weight = tf.constant([[1, 1, 1]], dtype=tf.float32)

# calculation
sequence_loss1 = tf.contrib.seq2seq.sequence_loss(logits=prediction1, targets=y_data, weights=weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(logits=prediction2, targets=y_data, weights=weights)

# initialize network
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print("Loss1: ", sequence_loss1.eval(session=sess))
print("Loss2: ", sequence_loss2.eval(session=sess))

Loss1:  0.513015
Loss2:  0.371101


## RNN - Hi Hello Training

In [1]:
import tensorflow as tf
import numpy as np

# define RNN structure
hidden_size = 5      # output from the LSTM
input_dim = 5        # one-hot size
batch_size = 1       # one sentence
sequence_length = 6  # |ihello\ == 6

# input data conversion
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],  # h 0
              [0, 1, 0, 0, 0],  # i 1 
              [1, 0, 0, 0, 0],  # h 0
              [0, 0, 1, 0, 0],  # e 2
              [1, 0, 0, 1, 0],  # l 3
              [1, 0, 0, 1, 0]]] # l 3

y_data = [[1, 0, 2, 3, 3, 4]]   # ihello

# input = one-hot # output = integer 
X = tf.placeholder(tf.float32, [None, sequence_length, input_dim]) # see x_one_hot
Y = tf.placeholder(tf.int32, [None, sequence_length]) # see y_data

# make cell
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)

# What is it?
initial_state = cell.zero_state(batch_size, tf.float32)


outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(200):
        _loss, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        
        print(i, "loss: ", _loss, "prediction: ", result, "true Y: ", y_data, "Prediction str: ", ''.join(result_str) )


0 loss:  1.60222 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
1 loss:  1.52947 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
2 loss:  1.47063 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
3 loss:  1.4093 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
4 loss:  1.35425 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
5 loss:  1.31166 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
6 loss:  1.28564 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
7 loss:  1.2436 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
8 loss:  1.21094 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
9 loss:  1.17231 prediction:  [[3 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  llllll
10

109 loss:  0.637543 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
110 loss:  0.637347 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
111 loss:  0.637161 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
112 loss:  0.636982 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
113 loss:  0.636806 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
114 loss:  0.636633 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
115 loss:  0.636462 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
116 loss:  0.636294 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
117 loss:  0.636131 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] Prediction str:  ihello
118 loss:  0.635972 prediction:  [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3